In [126]:
# All Required imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

In [127]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [175]:
def preProcessData(data):
    test_df = data.drop(['Ticket','Name','PassengerId','Cabin'], axis=1)
    test_df['Female'] = test_df['Sex'] == 'female'
    test_df = test_df.drop('Sex', axis=1)
    test_df['Embarked'] = test_df['Embarked'] == 'S'
    test_df = test_df.dropna()
    return test_df

In [211]:
class TitanicDataset(Dataset):
    def __init__(self, data_set):
        self.data_set = data_set
    def __len__(self):
        return len(self.data_set)

    def __getitem__(self, idx):
        data = torch.tensor(self.data_set.loc[idx, test_df.columns != 'Survived'], dtype=torch.float32)
        label = nn.functional.one_hot(torch.tensor(self.data_set.loc[idx, test_df.columns == 'Survived'], dtype=torch.int64),num_classes=2)
        return data, label

In [212]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.simple_stack = nn.Sequential(
            nn.Linear(7, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 2),
        )
        
    def forward(self, x):
        x = self.simple_stack(x)
        return x

In [213]:
num_epochs = 5
batch_size = 2
lr = 0.005

In [214]:
ptrain_data = TitanicDataset(preProcessData(train_data))
ptest_data = TitanicDataset(preProcessData(test_data))
train_dataloader = DataLoader(ptrain_data, batch_size=1, shuffle=True)
test_dataloader = DataLoader(ptest_data, batch_size=4, shuffle=True)

In [ ]:
model = NeuralNetwork().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

for batch, (X, y) in enumerate(ptrain_data):
    # Compute prediction and loss
    pred = model(X.to(device))
    print(pred)
    print(y[0].to(device))
    loss = loss_fn(pred, y[0].to(device).type(torch.float32))

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 10 == 0:
        loss, current = loss.item(), batch * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

